In [50]:
import pandas as pd


In [51]:
df = pd.read_csv('requests.csv')

In [52]:
df.head()

,Request_ID,Client_IP,Country_Name,Gender,Age,Income,Time_of_Request,Requested_File,Is_Banned
0,84538,103.32.215.163,Trinidad and Tobago,Male,56-65,150k-250k,17:00:00,8555.html,0
1,84539,103.32.215.163,Trinidad and Tobago,Male,56-65,150k-250k,17:00:00,8555.html,0
2,84540,231.82.193.108,Egypt,Female,17-25,60k-100k,05:00:00,1148.html,0
3,84541,231.82.193.108,Egypt,Female,17-25,60k-100k,05:00:00,1148.html,0
4,84542,133.210.103.15,Brunei,Female,0-16,0-10k,20:00:00,3331.html,0


# Predicting Country 

In [53]:
# preprocess IP column
new_columns = df['Client_IP'].str.split('.', expand=True)
df[['client_ip_part1', 'client_ip_part2', 'client_ip_part3', 'client_ip_part4']] = new_columns

In [54]:
df['client_ip_part1'] = df['client_ip_part1'].astype(int)
df['client_ip_part2'] = df['client_ip_part2'].astype(int)
df['client_ip_part3'] = df['client_ip_part3'].astype(int)
df['client_ip_part4'] = df['client_ip_part4'].astype(int)

In [6]:
# train test split
from sklearn.model_selection import train_test_split
X = df[['client_ip_part1', 'client_ip_part2', 'client_ip_part3', 'client_ip_part4']]
y = df['Country_Name']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [9]:
preds = knn.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.9324706705242743

KNN gets 93.2% accuracy, lets try a random forest, svc, logistic reg, 

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)



RandomForestClassifier()

In [13]:
preds = rf.predict(X_test)
accuracy_score(preds, y_test)

1.0

In [14]:
# sv = SVC()
# sv.fit(X_train, y_train)
# preds = sv.predict(X_test)
# accuracy_score(preds, y_test)

In [15]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
preds = dt.predict(X_test)
accuracy_score(y_test, preds)

1.0

# Predicting income

In [16]:
df.head()

,Request_ID,Client_IP,Country_Name,Gender,Age,Income,Time_of_Request,Requested_File,Is_Banned,client_ip_part1,client_ip_part2,client_ip_part3,client_ip_part4
0,84538,103.32.215.163,Trinidad and Tobago,Male,56-65,150k-250k,17:00:00,8555.html,0,103,32,215,163
1,84539,103.32.215.163,Trinidad and Tobago,Male,56-65,150k-250k,17:00:00,8555.html,0,103,32,215,163
2,84540,231.82.193.108,Egypt,Female,17-25,60k-100k,05:00:00,1148.html,0,231,82,193,108
3,84541,231.82.193.108,Egypt,Female,17-25,60k-100k,05:00:00,1148.html,0,231,82,193,108
4,84542,133.210.103.15,Brunei,Female,0-16,0-10k,20:00:00,3331.html,0,133,210,103,15


In [17]:
df['Income'].value_counts()

Income
100k-150k    12164
0-10k        12126
20k-40k      11942
40k-60k      11906
60k-100k     11838
10k-20k      11836
150k-250k    11782
250k+        11716
Name: count, dtype: int64

In [55]:
# ordinal encoding for ordered categorical vars, income, age, 
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[['0-10k', '10k-20k', '20k-40k', '40k-60k', '60k-100k', '100k-150k' ,'150k-250k' ,'250k+']])
df['Income_encoded'] = ordinal_encoder.fit_transform(df[['Income']])

ordinal_encoder = OrdinalEncoder(categories=[['0-16', '17-25', '26-35', '36-45', '46-55', '56-65' ,'66-75' ,'76+']])
df['Age_encoded'] = ordinal_encoder.fit_transform(df[['Age']])


In [56]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Gender_encoded'] = le.fit_transform(df['Gender'])

In [57]:
df['Age_encoded'].value_counts()

Age_encoded
3.0    12166
2.0    12102
1.0    11954
6.0    11946
7.0    11844
0.0    11812
4.0    11792
5.0    11694
Name: count, dtype: int64

In [58]:
X = df[['client_ip_part1', 'client_ip_part2', 'client_ip_part3', 'client_ip_part4', 'Age_encoded', 'Gender_encoded']]
y = df['Income_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# KNN

In [48]:
knn = KNeighborsClassifier(n_neighbors= 5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [49]:
preds = knn.predict(X_test)
accuracy_score(preds, y_test)

0.20319206434998252

Use gridsearch:

In [22]:
from sklearn.model_selection import GridSearchCV

knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}


knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(estimator=knn, param_grid=knn_params, cv=5, verbose=1, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)


print("Best parameters for KNN:", grid_search_knn.best_params_)
print("Best score for KNN:", grid_search_knn.best_score_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters for KNN: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Best score for KNN: 0.591853543458435


# Random Forest

In [63]:
rf = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
accuracy_score(preds, y_test)

0.13117985565764792

Use gridsearch:

In [54]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30]
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=rf_params, cv=5, verbose=1, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)


print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best score for Random Forest:", grid_search_rf.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters for Random Forest: {'max_depth': None, 'n_estimators': 200}
Best score for Random Forest: 0.5917596651786268


In [23]:
# Best parameters for KNN: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
goodknn = KNeighborsClassifier(n_neighbors=3, metric='manhattan', weights='distance')
goodknn.fit(X_train, y_train)
preds = goodknn.predict(X_test)

In [24]:
accuracy_score(preds, y_test)

0.7050519823228308

In [59]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
preds = dt.predict(X_test)
accuracy_score(y_test, preds)

0.7020316027088036

In [27]:
preds

array([7., 5., 4., ..., 7., 3., 2.])

In [28]:
y_test

9979     7.0
93862    2.0
12172    7.0
95187    3.0
67473    1.0
        ... 
13484    5.0
4790     4.0
50841    7.0
31641    5.0
53700    2.0
Name: Income_encoded, Length: 31453, dtype: float64

In [30]:
sum((preds - y_test)**2)

116200.0

In [34]:
from sklearn.model_selection import RandomizedSearchCV

dtc = DecisionTreeClassifier(random_state=42)

# Create a dictionary to try different hyperparameters
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50]
}

# Initialize GridSearchCV with the Decision Tree Classifier and the desired parameters
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print out the best parameters
print(f"Best parameters found: {grid_search.best_params_}")

# Use the best estimator to make predictions
best_dtc = grid_search.best_estimator_
y_pred = best_dtc.predict(X_test)

# Evaluate the best model's performance
from sklearn.metrics import accuracy_score
print(f"Accuracy of the best classifier: {accuracy_score(y_test, y_pred)}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters found: {'criterion': 'gini', 'max_depth': None, 'splitter': 'random'}
Accuracy of the best classifier: 0.7017772549518329


In [36]:
# !pip install xgboost
import xgboost as xgb

In [40]:
df['client_ip_part1'] = df['client_ip_part1'].astype(int)
df['client_ip_part2'] = df['client_ip_part2'].astype(int)
df['client_ip_part3'] = df['client_ip_part3'].astype(int)
df['client_ip_part4'] = df['client_ip_part4'].astype(int)

In [46]:
X = df[['client_ip_part1', 'client_ip_part2', 'client_ip_part3', 'client_ip_part4', 'Age_encoded', 'Gender_encoded']]
y = df['Income_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [47]:
df['Income_encoded']

0        6.0
1        6.0
2        4.0
3        4.0
4        0.0
        ... 
95305    1.0
95306    0.0
95307    0.0
95308    4.0
95309    4.0
Name: Income_encoded, Length: 95310, dtype: float64

In [48]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)

# Evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100.0}%")

Accuracy: 27.358280609162875%


In [39]:
df['client_ip_part1']

0        103
1        103
2        231
3        231
4        133
        ... 
95305     29
95306    214
95307    214
95308    177
95309    177
Name: client_ip_part1, Length: 95310, dtype: object